In [8]:
import tensorflow as tf
import numpy as np
import PIL
from imgnet32y_RDAE import *
import matplotlib.pyplot as plt
from functools import wraps
from time import time

In [9]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap

In [31]:
IMG_SIZE = 32;
N_CHANNELS = 1;
ALPHA = 0.125;
batch_size = 32;
NUM_CLASSES = 10;
size = (IMG_SIZE,IMG_SIZE);

(ds_train, ds_test), ds_info = tfds.load(
    "imagenette", split=['train','validation'], with_info=True, as_supervised=True
)

In [3]:
def resize(image, label):
    resized = tf.image.resize(image, size)
    resized = tf.image.rgb_to_grayscale(resized)
    return resized, label
def onehot(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

In [44]:
ds_train = ds_train.map(resize, num_parallel_calls=8)
ds_train = ds_train.map(onehot, num_parallel_calls=8)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)

ds_test = ds_test.map(resize, num_parallel_calls=8)
ds_test = ds_test.map(onehot, num_parallel_calls=8)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [7]:
compressor,decompressor = load_pretrained_RDAE()

1/1 [==============================] - ETA: 0s - loss: 2262.4705 - distortion_loss: 0.9994 - rate_loss: 263.6897 - distortion_pass_through_loss: 0.9994 - rate_pass_through_loss: 263.6897

1/1 [==============================] - 4s 4s/step - loss: 2262.4705 - distortion_loss: 0.9994 - rate_loss: 263.6897 - distortion_pass_through_loss: 0.9994 - rate_pass_through_loss: 263.6897 - val_loss: 2256.1201 - val_distortion_loss: 0.9981 - val_rate_loss: 259.9435 - val_distortion_pass_through_loss: 0.9981 - val_rate_pass_through_loss: 259.9435


In [49]:
@timing
def encode():
    C = []
    for X,y in ds_train.take(-1):
        strings, entropies = compressor(X)
        C.append(strings)
    C = tf.reshape(tf.stack(C),(9440))
    np.save('imagenette_32y',np.array(C))
encode()

2023-04-18 13:48:54.269115: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [16]
	 [[{{node Placeholder/_4}}]]
2023-04-18 13:48:54.270159: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [16]
	 [[{{node Placeholder/_3}}]]


func:'encode' args:[(), {}] took: 4.8601 sec


In [69]:
@timing
def partial_decode():
    C = np.load('imagenette_32y.npy',allow_pickle=True)
    D = decompressor.entropy_model.decompress(C)
partial_decode()

TypeError: ContinuousBatchedEntropyModel.decompress() missing 1 required positional argument: 'broadcast_shape'

In [ ]:
decompressor.entropy_model.decompress(C,broadcast_shape=(128))

In [68]:
@timing
def full_decode():
    C = np.load('imagenette_32y.npy',allow_pickle=True)
    D = decompressor(C)
full_decode()

func:'full_decode' args:[(), {}] took: 0.0403 sec


In [ ]:
# 
# for i_example in range(5):
#     x = X[i_example].numpy()
#     x = tf.convert_to_tensor(x.astype(np.uint8))
#     display_tf(x)
#     display_tf(reconstructions[i_example])
#     pass

In [ ]:
MBNet = MobileNetV2(
    include_top=True,
    weights=None,
    alpha=ALPHA,
    classes=10,
    input_shape=(IMG_SIZE,IMG_SIZE,N_CHANNELS),
)
model = MBNet
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
print(model.layers[0].get_input_shape_at(0))
print(model.layers[-1].get_output_shape_at(0))
print(np.size(model.layers))

13% GPU utilization

In [ ]:
epochs = 30
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')